In [2]:
!pip install catboost

In [3]:
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score
import pickle
from catboost import CatBoostClassifier

In [4]:
import os
import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/doanthuan/viettelpay/master/util.py', 'util.py')

('util.py', <http.client.HTTPMessage at 0x7f157d780c50>)

In [5]:
from util import *

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_df = pd.read_csv("/content/drive/MyDrive/projects/viettelpay/code/data/data-3m.csv")
test_df = pd.read_csv("/content/drive/MyDrive/projects/viettelpay/code/data/data-202008.csv")

In [8]:
train_df.head()

,province_code,district_code,status_name,cust_name,msisdn,ben_cust_name,ben_msisdn,request_date,viettel_bank_code,channel_type_id,msisdn_channel,channel_code,staff_code,trans_amount,trans_fee,trans_type,process_code,channel_fee,shop_code,transfer_type,customer_fee,fee_partner,is_fraud
0,H031,H031007,NaN,NaN,000186ca85dea743c31b,NaN,NaN,2020-07-20T16:44:41.000+07:00,KPP,37.0,000186ca85dea743c31b,6100280135,6100280135,446151.0,909.0,3.0,W_KPP301,700.0,6100280135,NaN,0.0,1000.0,0
1,P210,P210009,Đã hoàn trả tiền vào TK BCCS,HA THI XUYEN,00018fc18afe4b9bffaa,HA NGOC LUYEN,00017da8a27b9b7974a4,2020-07-20T06:37:09.000+07:00,KPP,1.0,000176d2f9ca9c597448,1001344100_00772_DB,1001344100_00772_DB,4000000.0,12273.0,4.0,KPP501,10800.0,1001344100_VTP,BCCS,27000.0,0.0,0
2,H019,H019001,NaN,Triệu Thị Hon,000135da6ed6e6692327,Triệu Thị Hon,000135da6ed6e6692327,2020-07-20T13:41:46.000+07:00,VTT,164.0,0001e6d09b082d64e0fa,DIEPHTB_VTPOST_HGG,DIEPHTB_VTPOST_HGG,5000000.0,0.0,2.0,400100,7500.0,1800220009,NaN,0.0,0.0,0
3,H018,H018009,NaN,NaN,0001b6a32422e06151dd,NaN,NaN,2020-07-20T18:23:56.000+07:00,KPP,1.0,0001b6a32422e06151dd,1001530100_00433_DB,1001530100_00433_DB,248677.0,909.0,3.0,KPP301,700.0,1001530100_VTP,NaN,0.0,1000.0,0
4,N350,N350003,NaN,NaN,000110bbafc5de415ba3,NaN,NaN,2020-07-20T20:16:01.000+07:00,KPP,1.0,000110bbafc5de415ba3,1001240100_00647_DB,1001240100_00647_DB,461212.0,1000.0,3.0,KPP301,770.0,1001240100_VTP,NaN,0.0,1100.0,0


In [9]:
cat_features = ["province_code", "district_code","viettel_bank_code","channel_code"
                ,"staff_code","process_code","shop_code"]

train_df = preprocess_data(train_df, cat_features)
test_df = preprocess_data(test_df, cat_features)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
y_train = train_df['is_fraud']
X_train = train_df.drop(['is_fraud'], axis = 1)
y_test = test_df['is_fraud']
X_test = test_df.drop(['is_fraud'], axis = 1)
    
X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=0.2, random_state=42)

In [10]:
cat_model = CatBoostClassifier(
    iterations=4000,
    max_ctr_complexity=1,
    random_seed=0,
    od_type='Iter',
    od_wait=25,
    verbose=25,
    depth=6,
    task_type="GPU",
    loss_function="Logloss",
    eval_metric="AUC"
)

cat_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val)
)

Learning rate set to 0.017585
0:	learn: 0.8565025	test: 0.8557387	best: 0.8557387 (0)	total: 217ms	remaining: 14m 28s
25:	learn: 0.9886458	test: 0.9889411	best: 0.9889928 (16)	total: 5.41s	remaining: 13m 47s
50:	learn: 0.9899758	test: 0.9903359	best: 0.9903359 (50)	total: 10.7s	remaining: 13m 49s
75:	learn: 0.9910247	test: 0.9912522	best: 0.9912522 (75)	total: 16s	remaining: 13m 45s
100:	learn: 0.9912337	test: 0.9914524	best: 0.9914892 (96)	total: 21.3s	remaining: 13m 41s
125:	learn: 0.9913084	test: 0.9915131	best: 0.9915131 (125)	total: 26.6s	remaining: 13m 38s
150:	learn: 0.9915992	test: 0.9917961	best: 0.9917961 (150)	total: 31.9s	remaining: 13m 33s
175:	learn: 0.9918705	test: 0.9920460	best: 0.9920460 (175)	total: 37.2s	remaining: 13m 28s
200:	learn: 0.9920695	test: 0.9922327	best: 0.9922327 (200)	total: 42.5s	remaining: 13m 24s
225:	learn: 0.9922388	test: 0.9924075	best: 0.9924075 (225)	total: 47.9s	remaining: 13m 19s
250:	learn: 0.9923429	test: 0.9925200	best: 0.9925200 (250)	tot

In [11]:
# from sklearn.metrics import make_scorer, accuracy_score
# from sklearn.model_selection import train_test_split, GridSearchCV

# clf = CatBoostClassifier()

# params = {'iterations': [2000],
#           'depth': [6,8,10],
#           'loss_function': ['Logloss', 'CrossEntropy'],
#           'random_seed': [42],
#           'max_ctr_complexity':[1],
#           'od_type':['Iter'],
#           'od_wait':[25],
#           'verbose':[25],
#           'task_type':["GPU"]
#          }

# scorer = make_scorer(accuracy_score)
# clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)

# clf_grid.fit(X_train, y_train,  cat_features=cat_features, eval_set=(X_val, y_val))
# best_param = clf_grid.best_params_
# best_param

In [12]:
#save_model(cat_model, 'cat-model.saved')
pickle.dump(cat_model, open('cat-model.saved', 'wb'))

In [13]:
#loaded_cat_model = load_model('cat-model.saved')
loaded_cat_model = pickle.load(open('cat-model.saved', 'rb'))
#test_pred = loaded_cat_model.predict(X_test)

In [ ]:
eval_model(loaded_cat_model, X_test, y_test)

In [ ]:
#importances = loaded_cat_model.feature_importances_
importances = loaded_cat_model.get_feature_importance(type= "FeatureImportance")
importances

In [ ]:
# from multiprocessing import Pool
# import shap
# shap_values = loaded_cat_model.get_feature_importance(Pool(X_test, label=y_test,cat_features=cat_features), type="ShapValues")
# explainer = shap.Explainer(loaded_cat_model)
# shap_values = explainer(X_test)

# # visualize the first prediction's explanation
# shap.plots.waterfall(shap_values[0])

In [ ]:
feature_names = [
        "province_code",
        "district_code",
        "date_diff",
        "viettel_bank_code",
        "channel_type_id",
        "channel_code",
        "staff_code",
        "trans_amount",
        "trans_fee",
        "trans_type",
        "process_code",
        "channel_fee",
        "shop_code",
        "customer_fee",
        "fee_partner",
        "same_name",
        "same_phone",
        "same_phone_channel",
        "same_phone_channel_ben"
    ]
df_feature_importance = pd.DataFrame(loaded_cat_model.feature_importances_, index=feature_names, columns=['feature importance']).sort_values('feature importance', ascending=False)
df_feature_importance

In [ ]:
df_feature_importance.plot(kind='bar');

In [ ]:
def plot_feature_importance(model, features):
    feature_importances = pd.DataFrame(sorted(zip(model.feature_importances_, features)), columns=['Value', 'Feature'])
    plt.figure(figsize=(20,10))
    sns.barplot(x="Value", y="Feature", data=feature_importances.sort_values(by="Value", ascending=False).iloc[:50])
    plt.title("Feature Importance")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_feature_importance(loaded_cat_model, feature_names)